
# Term 1 Project 4 _FindingLanes

To find the lanes from the videos, the following procedure would be proceeded:
- define image process functions, 
- build image process pipeline
- run the pipeline on the videos 

1. Define line and image parameters class 
- Camera calibration
- Image distortion
- Image filtering by color & gradient threshold
- Image perspective transform to bird-view
- Polynomial fit lane line
- Visualization on test images for the function testing. 
- Finally, do visualization on vedios.

## 1. Define Functions

Classes and functions are built for later use in setting up pipeline.

0. Class: Line() and Image_Parameters()
1. Camera calibration
- Image distortion
- Image filtering by color & gradient threshold
- Image perspective transform to bird-view
- Polynomial fit lane line
- Visualization 


In [2]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os
import glob
%matplotlib inline

''' 1 Define class '''
class Line():
    def __init__(self):       
        self.fit = []     # Polynomial fit parameters [A,B,C] where (A*y**2+B*y+C=x)
        # the axis y of pixels which belong to lane 
        #self.y = []
        # the axis x of pixels which belong to lane 
        #self.x = []
        # the linear interpolation y 
        #self.ploty = []
        
        self.fitx = []    # The axis x of fitted lane line
        self.curverad_m = None # the curvature radius in meter, at the point close to car body

class Image_Parameters():
    def __init__(self):
        self.calibrated = None   # "ret, mtx, dist, rvecs, tvecs"
        self.M = []              # 3x3 perspective transform matrix
        self.Minv = []           # 3x3 reverse perspective transform matrix


''' 2 Calibration ''' 
def calibration(img_path, corner_y, corner_x):
    objpoints = []
    imgpoints = []
    
    # The should-be corner point coordinates
    objcorner = np.zeros((corner_y*corner_x,3), np.float32)
    objcorner[:, 0:2] = np.mgrid[0:corner_x, 0:corner_y].T.reshape(-1,2)
    
    # Capture image corner point coordinate
    img_cli = glob.glob(img_path)
    for idx, fname in enumerate(img_cli):
        img = mpimg.imread(fname)  
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (corner_x,corner_y), None)
        if ret == True: 
            objpoints.append(objcorner)
            imgpoints.append(corners)
    
    # Calibration
    one_img = mpimg.imread(img_cli[0])
    img_size = (one_img.shape[1], one_img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, img_size, None, None)
    
    return ret, mtx, dist, rvecs, tvecs

''' 3 Undistortion '''
# cv2.undistort(image, mtx, dist, None, mtx) 

''' 4 Image thresholding '''
def color_gradient_threshold(img, sx_thresh=(0, 255), sc_thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float32)
    H_channel, L_channel, S_channel = hls[:,:,0], hls[:,:,1], hls[:,:,2]

    sobel_x = cv2.Sobel(S_channel, cv2.CV_64F, 1, 0)
    sbs_sx = np.absolute(sobel_x)
    scaled_sx = np.uint8(255*sbs_sx/np.max(sbs_sx))
    # gradient thresholding on x axile
    sx_binary = np.zeros_like(scaled_sx)
    sx_binary[(scaled_sx > sx_thresh[0]) & (scaled_sx <= sx_thresh[1])] = 1
    # color thresholding on hls's S color
    sc_binary = np.zeros_like(S_channel)
    sc_binary[(S_channel > sc_thresh[0]) & (S_channel <= S_channel[1])] = 1
    # combined above two threshold into one colored image
    combined_binary = np.zeros_like(sx_binary)
    combined_binary[(sx_binary==1) | (sc_binary==1)] = 1
    
    return combined_binary   

#### 1.1 Function 2 & 3 test

In [ ]:
''' - Test: calibration and undistortion '''
# Calibration
ret, mtx, dist, rvecs, tvecs = calibration("camera_cal/*.jpg", 9, 6)

# Undistortion
for img in os.listdir("camera_cal/"):
    img_cli = mpimg.imread("camera_cal/"+img)
    undistort = cv2.undistort(img_cli, mtx, dist, None, mtx)
    
    # Visualize
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img_cli)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(undistort)
    ax2.set_title('Undistorted Image', fontsize=20)


In [3]:
''' 5 Perspective Transform '''
# Capture source corners from houghline 
def corners_from_houghlines(lines, y_min, y_max):
    i = yr_mid = xr_mid = s_r = b_r = 0      # y_mid and x_mid: midpoint of houghline
    j = yl_mid = xl_mid = s_l = b_l = 0      # s and b: slope and intercept of houghline
       
    # Seperate left and right lines
    for line in lines:
        for x1,y1,x2,y2 in line:             
            
            # Right Lane Section
            if 2>((x2-x1)/(y2-y1))>0: 
                i += 1
                yr_mid += (y2 + y1)/2        # accumulate the midpoint y              
                xr_mid += (x2 + x1)/2        # accumulate the midpoint x
                s_r += (x2-x1) / (y2-y1)     # accumulate the slope
                b_i = ((x2+x1) - (x2-x1)/(y2-y1)*(y2+y1))/ 2 
                if i == 1:                   # accumulate the intercept b
                    b_r = b_i
                else:
                    if abs(b_i - b_r/(i-1)) < 20: 
                        b_r = xr_mid - s_r/i * yr_mid
                    else:                    # throw away deviated lines  
                        i -= 1
                        yr_mid -= (y2 + y1)/2
                        xr_mid -= (x2 + x1)/2
                        s_r -= (x2-x1) / (y2-y1)    
            
            # Left Lane Section
            elif 0>((x2-x1)/(y2-y1))>-2: 
                j += 1
                yl_mid += (y2 + y1)/2
                xl_mid += (x2 + x1)/2
                s_l += (x2-x1) / (y2-y1)
                b_j = ((x2+x1) - (x2-x1)/(y2-y1)*(y2+y1))/ 2 
                if j == 1:
                    b_l = b_j
                else:                    
                    if abs(b_j - b_l/(j-1)) < 20:
                        b_l = xl_mid - s_l/j * yl_mid
                    else:                        
                        j -= 1
                        yl_mid -= (y2 + y1)/2
                        xl_mid -= (x2 + x1)/2
                        s_l -= (x2-x1) / (y2-y1)
    
    # Calculate lane corners [left_top, left_bottom, right_bottom, right_top]
    if j == 0:
        xl_top = xl_bottom = 0
        print("Error: Left lane not found.")
    else:
        xl_top = (s_l * y_min + b_l)/j
        xl_bottom = (s_l * y_max + b_l)/j
    
    if i == 0:
        xr_top = xr_bottom = 0
        print("Error: Right lane not found.")
    else:
        xr_top = (s_r * y_min + b_r)/i
        xr_bottom = (s_r * y_max + b_r)/i  
    
    corners = np.float32([[xl_top, y_min], [xl_bottom, y_max], 
                          [xr_bottom, y_max], [xr_top, y_min]])
    
    return corners     


# Input expecting combined_binary image (color channel=1)
def perspective_transform(img, test="off"): 
    # Mask image in black, except the triangle area where lane stays
    mask = np.zeros_like(img)   
    vertices = np.array([[(0,img.shape[0]),(img.shape[1]/2,img.shape[0]*.59),
                          (img.shape[1],img.shape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    
    # Generate the houghlines
    rho = 1
    theta = np.pi/360
    threshold = 25
    min_line_len = 1
    max_line_gap = 100
    
    hough_line = cv2.HoughLinesP(masked_img, rho, theta, threshold, np.array([]),
                                minLineLength=min_line_len, maxLineGap=max_line_gap)
    #
    for x1,y1,x2,y2 in hough_line[0]:
        cv2.line(masked_img,(x1,y1), (x2,y2), (0,255,0),2)
    #plt.imshow(masked_img)
    
    
    #
    # Capture source corners from houghline 
    corners = corners_from_houghlines(hough_line, y_min=img.shape[0]*0.625, 
                                     y_max=img.shape[0]*0.95)   
    # Perspective transform
    destinated = np.float32([[300,10],[300,700],[1100,700],[1100,10]]) # manual  
    M = cv2.getPerspectiveTransform(corners, destinated)
    Minv = cv2.getPerspectiveTransform(destinated, corners)
    
    
    #-----UPDATING: Visualization for testing
    if test == 'off':
        return M, Minv
    
    if test == 'on':
        img_size = (img.shape[1], img.shape[0])
        binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
        # draw lines on combined_binary
        img_out = np.dstack((img,img,img))*255
        cv2.polylines(img_out, np.int32([corners]), True, (255,0,0), 5)
        # draw lines on binary_warped
        warped_out = np.dstack((binary_warped,binary_warped,binary_warped))*255
        cv2.polylines(warped_out, np.int32([destinated]), True, (255,0,0), 5)
        
        return img_out, warped_out

# cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
# (M calculated once as assuming the road is flat and the camera perspective hasn't changed)


#### 1.2 Function 4 & 5 test

In [ ]:
''' - Test: image thresholding and perspective transform '''

sx_thresh = (20,120) # x gradient
sc_thresh = (170,255) # color

for img in os.listdir("test_images/"):
    image_origin = mpimg.imread("test_images/"+img)
    image = np.copy(image_origin)
        
    undistort = cv2.undistort(image, mtx, dist, None, mtx)
    combined_binary = color_gradient_threshold(undistort, sx_thresh, sc_thresh)
    combined_binary_size = (combined_binary.shape[1], combined_binary.shape[0])
    
    c_b_wLine, warped_wLine = perspective_transform(combined_binary, test='on')
    
    # Visualize undistortion
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,10))
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(undistort)
    ax2.set_title('Undistorted Image', fontsize=20)
    ax3.imshow(c_b_wLine, cmap='gray')
    ax3.set_title('Threshold & Line', fontsize=20)
    ax4.imshow(warped_wLine)
    ax4.set_title('Warped & Line', fontsize=20)
    #cv2.imwrite("output_images/"+img, undistort)


In [4]:
''' 6 Polynomial fit lane line '''

# Input expecting binary_warped image (color channel=1)
def line_fit(img, test='off'):
    # Restore the (y,x) of white color pixels
    nonzeroy = np.array(img.nonzero()[0])
    nonzerox = np.array(img.nonzero()[1]) 

    # Indices of pixels  
    left_lane_inds = []
    right_lane_inds = []
    
    # Parameters
    margin = 100
    minpix = 50
    # Prepare an image to draw the lines
    out_img = np.dstack((img,img,img))*255
    
    # Capture the lane line indices
    if left.fit==[] or right.fit==[]: 
        # Find the lanes' starting point:
        histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[0:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        # Slide the image 
        nwindows = 9
        window_height = np.int(img.shape[0] / nwindows)
        # x value of lane in each slide window
        leftx_current = leftx_base
        rightx_current = rightx_base        
        
        for window in range(nwindows):
            # Rectangle vertices
            win_y_low = img.shape[0] - (window+1) * window_height
            win_y_high = img.shape[0] - window * window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            if test == 'on':
                cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0,255,0), 2)
                cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0,255,0), 2) 
                 
            # Capture the lane pixels in the rectangle       
            good_left_inds = ((nonzeroy>=win_y_low)&(nonzeroy<win_y_high) & 
                              (nonzerox>=win_xleft_low)&(nonzerox<win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy>=win_y_low)&(nonzeroy<win_y_high) &
                               (nonzerox>=win_xright_low)&(nonzerox<win_xright_high)).nonzero()[0]

            left_lane_inds.append(good_left_inds)         
            right_lane_inds.append(good_right_inds)
            
            if len(good_left_inds) > minpix: # update x value of lane for next slide window
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
       
    # Capture the lane indices of continuous images
    else:
        # Capture the white pixels close to last image's lane
        left_lane_inds = (
            (nonzerox >= (left.fit[0]*nonzeroy**2 + left.fit[1]*nonzeroy + left.fit[2] - margin))
            & (nonzerox < (left.fit[0]*nonzeroy**2 + left.fit[1]*nonzeroy + left.fit[2] + margin)))
        right_lane_inds = (
            (nonzerox >= (right.fit[0]*nonzeroy**2 + right.fit[1]*nonzeroy + right.fit[2] - margin))
            & (nonzerox < (right.fit[0]*nonzeroy**2 + right.fit[1]*nonzeroy + right.fit[2] + margin)))
    
    # Capture the (y,x) of pixels which are lanes
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Polynomial fit
    left.fit = np.polyfit(lefty, leftx, 2)
    right.fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left.fitx = left.fit[0]*ploty**2 + left.fit[1]*ploty + left.fit[2]
    right.fitx = right.fit[0]*ploty**2 + right.fit[1]*ploty + right.fit[2]
    
    # Calculate the curvature radius
    y_eval = np.max(ploty)
    left_curverad = (1 + (2* left.fit[0]*y_eval + left.fit[1])**2
                    )**1.5 / np.absolute(2 * left.fit[0])
    right_curverad = (1 + (2* right.fit[0]*y_eval + right.fit[1])**2
                     )**1.5 / np.absolute(2 * right.fit[0])
    lane_width = (left.fit[0]*y_eval**2 + left.fit[1]*y_eval + left.fit[2]) 
    - (right.fit[0]*y_eval**2 + right.fit[1]*y_eval + right.fit[2])
    
    # Convert from pixel to meters
    ym_per_pix = 30 / img.shape[0]
    xm_per_pix = 3.7 / lane_width     
    l_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    r_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    left.curverad_m = (1 + (2*l_fit_m[0]*y_eval + l_fit_m[1])**2
                    )**1.5 / np.absolute(2 * l_fit_m[0])
    right.curverad_m = (1 + (2*r_fit_m[0]*y_eval + r_fit_m[1])**2
                    )**1.5 / np.absolute(2 * r_fit_m[0])
        
    y_eval_m = y_eval * ym_per_pix
    x_left_m = l_fit_m[0]*y_eval_m**2 + l_fit_m[1]*y_eval_m + l_fit_m[2]
    x_right_m = r_fit_m[0]*y_eval_m**2 + r_fit_m[1]*y_eval_m + r_fit_m[2]
    lane_width_m = x_left_m - x_right_m
    
    # Visualization for testing
    if test == 'off':
        return lane_width_m 

    if test == 'on':  
        # Result1: Lane parameters in meters
        print("Left & right curvature radius: %s m, %s m."%(left.curverad_m, right.curverad_m))
        print("Left & right lane distance:", lane_width_m)
    
        
        # Highlight the lane pixels        
        out_img[lefty, leftx] = [255,0,0]
        out_img[righty, rightx] = [0,0,255] 
        
        # Fill the lane line area
        window_img = np.zeros_like(out_img)
        
        left_lane_bound1 = np.array([np.transpose(np.vstack((left.fitx-margin, ploty)))])
        left_lane_bound2 = np.array([np.flipud(np.transpose(np.vstack((left.fitx+margin, ploty))))])
        left_lane_bound = np.hstack((left_lane_bound1, left_lane_bound2))
    
        right_lane_bound1 = np.array([np.transpose(np.vstack((right.fitx-margin, ploty)))])
        right_lane_bound2 = np.array([np.flipud(np.transpose(np.vstack((right.fitx+margin, ploty))))])
        right_lane_bound = np.hstack((right_lane_bound1, right_lane_bound2))

        cv2.fillPoly(window_img, np.int_([left_lane_bound]), (0,255,0))
        cv2.fillPoly(window_img, np.int_([right_lane_bound]), (0,255,0))
    
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        
        # Result2: Plot the image and the line
        #plt.imshow(result)
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        
        return result      


#### 1.3 Function 6 test

In [ ]:
''' - Test: polynomial fit lane line '''

sx_thresh = (20,120) # x gradient
sc_thresh = (170,255) # color

for img in os.listdir("test_images/"):
    image_origin = mpimg.imread("test_images/"+img)
    image = np.copy(image_origin)
        
    undistort = cv2.undistort(image, mtx, dist, None, mtx)
    combined_binary = color_gradient_threshold(undistort, sx_thresh, sc_thresh)
    combined_binary_size = (combined_binary.shape[1], combined_binary.shape[0])
    
    M, Minv = perspective_transform(combined_binary)
    binary_warped = cv2.warpPerspective(
        combined_binary, M, combined_binary_size, flags=cv2.INTER_LINEAR)
    
    left = Line()
    right = Line()
    warped_wLine = line_fit(binary_warped, test='on')
    
    
    
    # Visualize undistortion
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,10))
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(undistort)
    ax2.set_title('Undistorted Image', fontsize=20)
    ax3.imshow(binary_warped, cmap='gray')
    ax3.set_title('Threshold', fontsize=20)
    ax4.imshow(warped_wLine)
    ax4.set_title('Warped & Line', fontsize=20)


In [5]:
''' 7 Lane area graphing '''
# Expecting what image??? --binary_warped (why???)
# -------params.Minv convert the warped image to combined_binary. Not sure what's the impact.
def lane_zone_visualize(img):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    pts_left = np.array([np.transpose(np.vstack([left.fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right.fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, params.Minv, (img.shape[1], img.shape[0])) 
    
    return newwarp
    


#### 1.3 Function 7 test

In [ ]:
''' - Test: polynomial fit lane line '''

sx_thresh = (20,120) # x gradient
sc_thresh = (170,255) # color

for img in os.listdir("test_images/"):
    image_origin = mpimg.imread("test_images/"+img)
    image = np.copy(image_origin)
        
    undistort = cv2.undistort(image, mtx, dist, None, mtx)
    combined_binary = color_gradient_threshold(undistort, sx_thresh, sc_thresh)
    combined_binary_size = (combined_binary.shape[1], combined_binary.shape[0])
    
    params = Image_Parameters()
    params.M, params.Minv = perspective_transform(combined_binary)
    binary_warped = cv2.warpPerspective(
        combined_binary, params.M, combined_binary_size, flags=cv2.INTER_LINEAR)
    
    left = Line()
    right = Line()
    warped_wLine1 = line_fit(binary_warped, test='on')
    warped_wLine = lane_zone_visualize(binary_warped)
    result = cv2.addWeighted(undistort, 1, warped_wLine, 0.3, 0)
    
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
    ax1.imshow(undistort)
    ax1.set_title('Undistorted Image', fontsize=20)
    #ax2.imshow(combined_binary, cmap='gray')
    #ax2.set_title('Threshold Image', fontsize=20)
    #ax3.imshow(warped_wLine1, cmap='gray')
    #ax3.set_title('Perspective Transform', fontsize=20)
    ax2.imshow(result)
    ax2.set_title('Undistorted & Line', fontsize=20)


## 2. Build pipeline

In [6]:
''' 
These class objects should be defined 
before calling pipeline:
    left = Line()
    right = Line()
    params = Line_Parameters()

'''
def pipeline(img, test='off'):
    # Parameters
    #img_size = (image.shape[1], image.shape[0])
    sx_thresh = (20,120) # x gradient
    sc_thresh = (170,255) # color
    
    # Undistortion
    if params.calibrated == None:
        params.calibrated = calibration("camera_cal/*.jpg", 9, 6)
    ret, mtx, dist, rvecs, tvecs = params.calibrated  
    undistort = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Gradient & color threshold
    combined_binary = color_gradient_threshold(undistort, sx_thresh, sc_thresh)
    combined_binary_size = combined_binary.shape[1],combined_binary.shape[0]
    
    # Perspective transform
    if params.M==[] or params.Minv==[]:
        params.M, params.Minv = perspective_transform(combined_binary)
    binary_warped = cv2.warpPerspective(
        combined_binary, params.M, combined_binary_size, flags=cv2.INTER_LINEAR)
    
    # Lane line fit
    lane_width_m = line_fit(binary_warped)
    
    # Lane zone graphing
    warped_wLine = lane_zone_visualize(binary_warped)
      
    # Combine the result with the original image
    result = cv2.addWeighted(undistort, 1, warped_wLine, 0.3, 0)
    
    if test == 'off':
        return result
    else: 
        # Visualize       
        f, (ax1, ax2) = plt.subplots(1,2, figsize=(20,10))
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=20)
        ax2.imshow(result, cmap='gray')
        ax2.set_title('Result Image', fontsize=20)
 
        return result    

## 3 Test on Video

In [8]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define the class objects
left = Line()
right = Line()
params = Image_Parameters()

clip1 = VideoFileClip("project_video.mp4")#.subclip(0,5)
clip_result = clip1.fl_image(pipeline)

clip_save_addr = "output_images/project_video.mp4"
%time clip_result.write_videofile(clip_save_addr, audio=False)

print("The left lines's fit")

[MoviePy] >>>> Building video output_images/project_video.mp4
[MoviePy] Writing video output_images/project_video.mp4


100%|█████████▉| 1260/1261 [04:25<00:00,  4.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video.mp4 

CPU times: user 3min 43s, sys: 53.9 s, total: 4min 37s
Wall time: 4min 27s
The left lines's fit


In [7]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define the class objects
left = Line()
right = Line()
params = Image_Parameters()

clip1 = VideoFileClip("challenge_video.mp4")#.subclip(0,5)
clip_result = clip1.fl_image(pipeline)

clip_save_addr = "output_images/challenge_video.mp4"
%time clip_result.write_videofile(clip_save_addr, audio=False)

print("The left lines's fit")

Error: Right lane not found.
[MoviePy] >>>> Building video output_images/challenge_video.mp4
[MoviePy] Writing video output_images/challenge_video.mp4


100%|██████████| 485/485 [01:17<00:00,  6.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/challenge_video.mp4 

CPU times: user 1min 15s, sys: 12.9 s, total: 1min 27s
Wall time: 1min 19s
The left lines's fit


In [8]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define the class objects
left = Line()
right = Line()
params = Image_Parameters()

clip1 = VideoFileClip("harder_challenge_video.mp4")#.subclip(0,5)
clip_result = clip1.fl_image(pipeline)

clip_save_addr = "output_images/harder_challenge_video.mp4"
%time clip_result.write_videofile(clip_save_addr, audio=False)

print("The left lines's fit")

[MoviePy] >>>> Building video output_images/harder_challenge_video.mp4
[MoviePy] Writing video output_images/harder_challenge_video.mp4


 84%|████████▎ | 1003/1200 [02:36<00:24,  7.91it/s]

TypeError: expected non-empty vector for x

The left lines's fit


#### 2.1 Pipeline test on image

In [ ]:
''' - Test: Pipeline '''
# Define the class objects
left = Line()
right = Line()
params = Image_Parameters()

for img in os.listdir("test_images/"):
    image_origin = mpimg.imread("test_images/"+img)
    image = np.copy(image_origin)
    result = pipeline(image, test='on')